In [ ]:
import requests
import xml.etree.ElementTree as ET
import json
import geopandas as gpd
from shapely.geometry import Polygon, Point, box
from shapely.ops import unary_union
import numpy as np
from rtree import index  # Spatial indexing
import ipyparallel as ipp

In [23]:
url = "https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLValue?year=2018&x=1551565.363&y=1909363.537"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML content
    root = ET.fromstring(response.content)

    # Extract the Result element
    result_element = root.find(".//Result")  # No namespace needed

    if result_element is not None:
        # Get the text content of the Result element
        result_string = result_element.text

        # Fix the dictionary format by adding quotes around keys
        # Replace single quotes with double quotes for JSON compatibility
        result_string = result_string.replace("{", "{\"").replace(":", "\":").replace(", ", ", \"")

        # Convert the fixed string to a dictionary using json.loads
        result_dict = json.loads(result_string)

        print(result_dict)
    else:
        print("Result element not found in the XML response.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

{'x': 1551565.363, 'y': 1909363.537, 'value': 1, 'category': 'Corn', 'color': '#FFD400'}


In [25]:
def get_cdl_value(year, x, y):
    """
    Fetches and parses the CDL value for a given year, x, and y coordinates.

    Args:
        year (int): The year of the data.
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        dict: A dictionary containing the result data.
              Returns None if the request fails or the Result element is not found.
    """
    # Construct the URL with the provided year, x, and y
    url = f"https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLValue?year={year}&x={x}&y={y}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML content
        root = ET.fromstring(response.content)

        # Extract the Result element
        result_element = root.find(".//Result")  # No namespace needed

        if result_element is not None:
            # Get the text content of the Result element
            result_string = result_element.text

            # Fix the dictionary format by adding quotes around keys
            # Replace single quotes with double quotes for JSON compatibility
            result_string = result_string.replace("{", "{\"").replace(":", "\":").replace(", ", ", \"")

            # Convert the fixed string to a dictionary using json.loads
            result_dict = json.loads(result_string)

            return result_dict
        else:
            print("Result element not found in the XML response.")
            return None
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [26]:
# Example usage
year = 2018
x = 1551565.363
y = 1909363.537

result = get_cdl_value(year, x, y)
if result:
    print(result)
else:
    print("No result found.")

{'x': 1551565.363, 'y': 1909363.537, 'value': 1, 'category': 'Corn', 'color': '#FFD400'}


In [32]:
SGMA = "../data/GWBasins.shp"
gdf = gpd.read_file(SGMA)
gdf.set_crs(epsg=3857, inplace=True)
gdf.set_index('OBJECTID', inplace=True)
gdf['Basin_Prefix'] = gdf['Basin_Numb'].str.split('-').str[0].astype(int)
gdf_5021 = gdf[gdf['Basin_Numb'] == '5-021']
combined_polygon = unary_union(gdf_5021.geometry)

In [ ]:
combined_polygon = gpd.GeoSeries([combined_polygon], crs='EPSG:3857').to_crs('EPSG:32610').geometry[0]

# Get the bounds of the combined polygon
xmin, ymin, xmax, ymax = combined_polygon.bounds

# Create a grid of 100m x 100m squares
x = np.arange(xmin, xmax, 100)
y = np.arange(ymin, ymax, 100)

print("grid created")

grid_polygons = []
for xi in range(len(x) - 1):
    for yi in range(len(y) - 1):
        grid_polygons.append(box(x[xi], y[yi], x[xi + 1], y[yi + 1]))

print("boxes created")

# Create a GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame({'geometry': grid_polygons}, crs='EPSG:32610')

print("grid_gdf created")

# Intersect the grid with the combined polygon
intersected_gdf = gpd.overlay(grid_gdf, gpd.GeoDataFrame({'geometry': [combined_polygon]}, crs='EPSG:32610'), how='intersection')

grid created
boxes created
grid_gdf created
